# Mapping LineStrings

In [ ]:
import duckdb as ddb
ddb.load_extension('spatial')

In [ ]:
#parquet = r"https://overturemapswestus2.blob.core.windows.net/release/2023-07-26-alpha.0/theme%3Dadmins/type%3DadministrativeBoundary/20230725_211237_00132_5p54t_fef965a3-b003-48f8-8188-1c8c7b233ccb"
parquet = 'admin_boundary.parquet'

# Drop table admin_boundary if it exists
ddb.sql("DROP TABLE IF EXISTS admin_boundary")

#ddb.sql(f"CREATE TABLE admin_boundary AS SELECT * FROM read_parquet('{url}')")
#ddb.sql(f"CREATE TABLE admin_boundary AS SELECT *, ST_GeomFromWKB(geometry) AS geom FROM read_parquet('{url}')")
ddb.sql(f"CREATE TABLE admin_boundary AS SELECT id, geometry, ST_GeomFromWKB(geometry) AS geom, ST_AsGeoJSON(ST_GeomFromWKB(geometry)) AS json FROM read_parquet('{parquet}')")

In [ ]:
ddb.table('admin_boundary')

For folium, we want the geometry in a dictionary

In [ ]:
df = ddb.sql("SELECT json FROM admin_boundary").df()

In [ ]:
import json, geojson
import pandas as pd


# Import required modules
import json, geojson
import pandas as pd

# Convert the JSON strings in the 'json' column to Python objects
df['json'] = df['json'].apply(lambda x: json.loads(x))

# Create an empty list to store your GeoJSON features
features = []

# Iterate over df and create a GeoJSON feature for each row
for row in df.itertuples():
    # Create a GeoJSON feature for the current row
    feature = geojson.Feature(geometry=row.json, properties={})
    # Add the feature to the list of features
    features.append(feature)

# Create a GeoJSON feature collection from the list of features
feature_collection = geojson.FeatureCollection(features)

# Print the resulting GeoJSON feature collection
#print(geojson.dumps(feature_collection))

In [ ]:
#df

## Folium

In [ ]:
import folium

# Create a map object
m = folium.Map(location=[45.5236, -122.6750])

folium.GeoJson(feature_collection).add_to(m)

# Fit to the bounds of the data
m.fit_bounds(m.get_bounds())

In [ ]:
m

## Leafmap.leafmap

In [ ]:
import leafmap.leafmap as leafmap

m = leafmap.Map()
m.add_geojson(feature_collection, layer_name="GeoJSON", zoom_to_layer=True)

In [ ]:
m

## Leafmap.foliumap

In [ ]:
import leafmap.foliumap as leafmap

# Get the bounds of the GeoJSON file
#bounds = leafmap.get_geojson_bounds(feature_collection)

m = leafmap.Map()
m.add_geojson(feature_collection, layer_name="GeoJSON")

In [ ]:
m